In [ ]:
import pandas as pd
import numpy as np

AG = pd.read_csv('../data/B1_FavGames.csv')
AGP = AG[AG['Year']==2008][AG['Game']>=1]
AGPF = AGP.drop(['AUSeed','AUTeam','AUScore','Uti'],axis=1)
AGPU = AGP.drop(['AFSeed','AFTeam','AFScore','Fti'],axis=1)
AGPF['STS'] = AGPF['AFSeed'].astype(str)+' '+AGPF['AFTeam']+' '+AGPF['AFScore'].astype(str)
AGPF = AGPF.drop(['AFSeed','AFTeam','AFScore'],axis=1).rename(columns={"Fti": "ti"})
AGPU['STS'] = AGPU['AUSeed'].astype(str)+' '+AGPU['AUTeam']+' '+AGPU['AUScore'].astype(str)
AGPU = AGPU.drop(['AUSeed','AUTeam','AUScore'],axis=1).rename(columns={"Uti": "ti"})
AGC = pd.concat([AGPF,AGPU]).sort_values(by=['Game','ti'])
AGC.reset_index(inplace=True)
R1W = np.array(AGC[AGC['Game']<=16]['STS'])
R1E = np.array(AGC[(AGC['Game']>=17) & (AGC['Game']<=32)]['STS'])
R2W = np.array(AGC[(AGC['Game']>=33) & (AGC['Game']<=40)]['STS'])
B16 = np.full((16),'',dtype=object)
R2W = np.insert(B16, np.arange(len(R2W)),R2W)
df = pd.DataFrame({'R1W':R1W,'R2W':R2W,'R1E':R1E})

In [ ]:
Champ = AGP.iloc[-1,]
if Champ['AFScore']>Champ['AUScore']:
    winner = Champ['AFTeam']
else:
    winner = Champ['AUTeam']
winner

In [ ]:
br_headers = ['Round 1 W','Round 2 W','Round 3 W','Round 4 W','Final Four W','Championship',
              'Final Four E','Round 4 E','Round 3 E','Round 2 E','Round 1 E']
df = pd.DataFrame(columns = br_headers)
for x in br_headers:
    for y in range(64):
        df.loc[y,x]=''
for y in range(64):
    df.loc[y,'Round 1 W']= AGC.loc[y,'STS']
for y in range(0,63,2):
    df.loc[y,'Round 2 W']= AGC.loc[y/2+64,'STS']
df

In [ ]:
import pandas as pd

AG = pd.read_csv('../data/B1_FavGames.csv')


LG = AG[AG['Round']=='6']
CS = pd.DataFrame({'Round','Seed'})
for x in range(0,36):
    if LG.iloc[x]["AFScore"]>LG.iloc[x]["AUScore"]:
        ws = LG.iloc[x]["AFSeed"]
    else:
        ws = LG.iloc[x]["AUSeed"]
    CS.loc[x,'Round'] = 'W'
    CS.loc[x,'Seed'] = ws
cs = pd.crosstab(index=CS['Seed'],columns=CS['Round'])
fs = pd.crosstab(index=AG['AFSeed'],columns=AG['Round'])
us = pd.crosstab(index=AG['AUSeed'],columns=AG['Round'])
ufs = fs.add(us, fill_value=0).astype(int).drop(['PI'],axis=1)
sh = ufs.add(cs, fill_value=0).fillna(0).astype(int)

sh



In [ ]:
import pandas as pd
import numpy as np
AG = pd.read_csv('../data/B1_FavGames.csv')

# table of wins
fw = pd.crosstab(index=AG['AFTeam'],columns=AG['Year'])
uw = pd.crosstab(index=AG['AUTeam'],columns=AG['Year'])
cw = fw.add(uw, fill_value=0).fillna(0).astype(int) - 1
cw = cw.replace(-1, '')


# table of seeds
AG1 = AG[AG['Round']=='1']
fs = AG1.pivot(index='AFTeam', columns='Year', values = 'AFSeed').fillna(0).astype(int)
us = AG1.pivot(index='AUTeam', columns='Year', values = 'AUSeed').fillna(0).astype(int)
cs = fs.add(us, fill_value=0).fillna(0).astype(int)
cs = cs.replace(0, '')

# table of expected wins
sh = pd.read_csv('../data/c3_SeedHistory.csv')
ew = pd.DataFrame({'Seed':np.arange(1,17),'Exp Wins':sh['Exp Wins']})

ewty = cs.replace(list(ew['Seed']),list(ew['Exp Wins']))

ewz = (ewty * -1).replace(r'^\s*$', np.nan, regex=True).fillna(0)
cwz = cw.replace(r'^\s*$', np.nan, regex=True).fillna(0)
pase = cwz.add(ewz).replace(0, np.nan)
pase['PASE']=pase.mean(axis=1)







In [ ]:
cw.astype(str)

In [ ]:
# practice for e_teamrank.py

import pandas as pd

pd.set_option('display.precision',3)
pd.options.display.float_format = '{:,.2f}'.format

TR = pd.read_csv('../data/B3_KPBPIBR.csv')
pase = pd.read_csv('../data/d3_PASE.csv')
pase.columns =['Team', 'PASE']
TRP = TR.merge(pase, on='Team', how = 'left')


In [ ]:
TRP

In [143]:
import pandas as pd
FUP = pd.read_csv("../data/f1_FUP.csv").fillna(0)
#FUP.info()
#FUP.isnull().sum().sum()
FUPN = FUP.select_dtypes(exclude=['object'])

# Split the dataframe into x and y
expl = FUPN.drop(['AFScore', 'AUScore'], axis=1)
respF = FUPN[['Year','AFScore']]
respU = FUPN[['Year','AUScore']]

In [181]:
from random import choice
#p_year = choice([i for i in range(2008,2021) if i not in [2020]])
p_year = 2012
print(p_year)

X_train = expl[expl['Year'] != p_year]
y_train_F = respF[respF['Year'] != p_year]['AFScore'].to_numpy() 
y_train_U = respU[respU['Year'] != p_year]['AUScore'].to_numpy() 
X_test = expl[expl['Year'] == p_year]
y_test_F = respF[respF['Year'] == p_year]['AFScore'].to_numpy()
y_test_U = respU[respU['Year'] == p_year]['AUScore'].to_numpy()

2012


In [182]:
from sklearn.linear_model import ElasticNet

#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train_s = scaler.transform(X_train)
enet = ElasticNet(alpha=.2,l1_ratio=.5)
enet.fit(X_train,y_train_F)
#scaler.fit(X_test)
#X_test_s = scaler.transform(X_test)
y_pred_F = enet.predict(X_test)
enet.fit(X_train,y_train_U)
y_pred_U = enet.predict(X_test)


In [183]:
actual=(y_test_F - y_test_U >0)

In [184]:
Pred=(y_pred_F - y_pred_U >0)

In [185]:
import numpy as np
print(p_year)
np.sum(actual == Pred)

2012


57

In [186]:
FUT = FUP[FUP['Year']==p_year]
FUT = FUT.iloc[:,0:10]
FUT['PFScore'] = y_pred_F
FUT['PUScore'] = y_pred_U
FUT['ESPN'] = (((FUT['AFScore'])>=(FUT['AUScore']))==((FUT['PFScore'])>=(FUT['PUScore']))).astype('uint8')*10*2**(FUT['Round']-1)
FUT['ESPN'].sum()

1850